In [1]:
##I'm going to try and read in a small-seized data file (100s of MB) about triathlon and analyze it
##Analytic Questions:
##What is the average time for the run in a Sprint triathlon for a 46-year-old male?
##What is the average time for the bike in a Sprint triathlon for a 46-year-old male?
##What is the average time for the swim in a Sprint triathlon for a 46-year-old male?

In [2]:
##import packages
import pandas as pd

In [4]:
##get data into a data object
# Assuming your CSV file is named 'triathlon_results.csv'
df = pd.read_csv('postgres_public_tristat_stat.csv')
print(df.head(3))

                                   event_link gender  \
0  /rus/result/ironman/ireland-cork/full/2019      M   
1  /rus/result/ironman/ireland-cork/full/2019      F   
2  /rus/result/ironman/ireland-cork/full/2019      M   

                     person_link person_flag        person_name  \
0    /irl/profile/halliwell-mark         IRL    Halliwell, Mark   
1      /usa/profile/harris-polly         USA      Harris, Polly   
2  /fra/profile/peugeot-rodolphe         FRA  Peugeot, Rodolphe   

  person_event_group person_event_swim_time_text person_event_t1_time_text  \
0             M45-49                        0:00                      0:00   
1             F50-54                        0:00                      0:00   
2             M25-29                        0:00                      0:00   

  person_event_cycle_time_text person_event_t2_time_text  \
0                      7:52:05                     19:00   
1                      7:58:53                     16:37   
2            

In [5]:
# Assuming df is your DataFrame and the column is named 'url_column'
# Split the string by forward slash and get the word after the fifth forward slash
df['word_after_fifth_slash'] = df['event_link'].str.split('/').str[5]

# Print the unique values
unique_words = df['word_after_fifth_slash'].unique()
print(unique_words)

['full' 'half' 'olympic' 'sprint' 'supersprint']


In [6]:
# Assuming df is your DataFrame
# Create a boolean mask to filter rows where the word is 'sprint' and swim_time has less than or equal to 5 characters
# add a couple clauses to only include swims and runs under an hour
# these clauses are because I'm having a hard time processing times that are an hour or more. 
mask = (df['event_link'].str.split('/').str[5] == 'sprint') & (df['person_event_swim_time_text'].str.len() <= 5) & (df['person_event_cycle_time_text'].str.len() <= 5) & (df['person_event_run_time_text'].str.len() <= 5)

# Apply the mask to the DataFrame
sprint_df = df[mask]

# Now, sprint_df contains only rows where the word is 'sprint' and swim, cycle, and run_time has less than or equal to 5 characters

In [7]:
# Assuming your DataFrame is named sprint_df and you want to remove columns 'event_link', 'person_link' and 'person_name'
columns_to_remove = ['event_link','person_link','person_name']
df = sprint_df.drop(columns=columns_to_remove)

In [8]:
##more clean up, change the column names
# Assuming your DataFrame is named df and you want to change column names
new_column_names = {'person_flag': 'flag', 'person_event_group': 'age','person_event_swim_time_text':'swim_time','person_event_t1_time_text':'t1_time','person_event_cycle_time_text':'cycle_time','person_event_t2_time_text':'t2_time','person_event_run_time_text':'run_time','person_event_finish_time_text':'finish_time'}

df = df.rename(columns=new_column_names)
print(df.head(3))

     gender flag     age swim_time t1_time cycle_time t2_time run_time  \
5501      M  RUS  M35-39     11:33    0:39      33:11    0:45    18:13   
5502      M  RUS  M35-39     14:23    0:30      31:31    0:36    18:52   
5503      M  RUS  M40-44     14:15    0:35      31:36    0:42    19:38   

     finish_time word_after_fifth_slash  
5501     1:04:21                 sprint  
5502     1:05:52                 sprint  
5503     1:06:46                 sprint  


In [9]:
##filter the data
# Assuming your dataset has columns like 'Age', 'Gender', 'RunTime'
filtered_data = df[(df['age'] == 'M45-49') & (df['gender'] == 'M')]
print(filtered_data.head(3))

     gender flag     age swim_time t1_time cycle_time t2_time run_time  \
5507      M  RUS  M45-49     13:25    0:48      32:16    0:48    22:05   
5513      M  RUS  M45-49     14:23    0:51      35:00    0:51    21:05   
5514      M  RUS  M45-49     16:01    0:40      34:09    0:40    20:45   

     finish_time word_after_fifth_slash  
5507     1:09:21                 sprint  
5513     1:12:10                 sprint  
5514     1:12:15                 sprint  


In [10]:
# Assuming filtered_data is your DataFrame and 'swim_time' is the column containing times with colons
filtered_data['swim_time'] = filtered_data['swim_time'].str.replace(':', '')

# Assuming filtered_data is your DataFrame and 'cycle_time' is the column containing times with colons
filtered_data['cycle_time'] = filtered_data['cycle_time'].str.replace(':', '')

# Assuming filtered_data is your DataFrame and 'run_time' is the column containing times with colons
filtered_data['run_time'] = filtered_data['run_time'].str.replace(':', '')



# Now, filtered_data will have colons removed from each value in the column
print(filtered_data.head(4))

     gender flag     age swim_time t1_time cycle_time t2_time run_time  \
5507      M  RUS  M45-49      1325    0:48       3216    0:48     2205   
5513      M  RUS  M45-49      1423    0:51       3500    0:51     2105   
5514      M  RUS  M45-49      1601    0:40       3409    0:40     2045   
5525      M  RUS  M45-49      2120    0:00       3424    0:46     2101   

     finish_time word_after_fifth_slash  
5507     1:09:21                 sprint  
5513     1:12:10                 sprint  
5514     1:12:15                 sprint  
5525     1:17:31                 sprint  


/tmp/ipykernel_54/416027161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['swim_time'] = filtered_data['swim_time'].str.replace(':', '')
/tmp/ipykernel_54/416027161.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['cycle_time'] = filtered_data['cycle_time'].str.replace(':', '')
/tmp/ipykernel_54/416027161.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [11]:
# Filter out rows where 'swim_time' is not equal to zero
filtered_data = filtered_data[filtered_data['swim_time'] != '0']

# Filter out rows where 'cycle_time' is not equal to zero
filtered_data = filtered_data[filtered_data['cycle_time'] != '0']

# Filter out rows where 'run_time' is not equal to zero
filtered_data = filtered_data[filtered_data['run_time'] != '0']

In [12]:
##create swim average
average_swim_time = filtered_data['swim_time'].mean()
print(average_swim_time)

##create cycle average
average_cycle_time = filtered_data['cycle_time'].mean()
print(average_cycle_time)

##create run average
average_run_time = filtered_data['run_time'].mean()
print(average_run_time)

inf
inf
inf


In [13]:
# Assuming filtered_data is your DataFrame
# We are going to fix whatever is causing the inf by converting to numeric

# Convert 'swim_time' to numeric, replacing non-numeric values with NaN
filtered_data['swim_time'] = pd.to_numeric(filtered_data['swim_time'], errors='coerce')
# Drop rows with NaN values in 'swim_time'
filtered_data = filtered_data.dropna(subset=['swim_time'])

# Convert 'cycle_time' to numeric, replacing non-numeric values with NaN
filtered_data['cycle_time'] = pd.to_numeric(filtered_data['cycle_time'], errors='coerce')
# Drop rows with NaN values in 'cycle_time'
filtered_data = filtered_data.dropna(subset=['cycle_time'])

# Convert 'run_time' to numeric, replacing non-numeric values with NaN
filtered_data['run_time'] = pd.to_numeric(filtered_data['run_time'], errors='coerce')
# Drop rows with NaN values in 'run_time'
filtered_data = filtered_data.dropna(subset=['run_time'])

In [14]:
# Now that the column is changed to numeric values, the 0's can be removed
# Filter out rows where 'swim_time' is not equal to zero
filtered_data = filtered_data[filtered_data['swim_time'] != '0']
filtered_data = filtered_data[filtered_data['cycle_time'] != '0']
filtered_data = filtered_data[filtered_data['run_time'] != '0']

In [24]:
##create and format averages
#swim
total_swim_seconds = int(filtered_data['swim_time'].mean())
formatted_swim_time = f'{total_swim_seconds // 100}:{total_swim_seconds % 100:02d}'
#cycle
total_cycle_seconds = int(filtered_data['cycle_time'].mean())
formatted_cycle_time = f'{total_cycle_seconds // 100}:{total_cycle_seconds % 100:02d}'
#run
total_run_seconds = int(filtered_data['run_time'].mean())
formatted_run_time = f'{total_run_seconds // 100}:{total_run_seconds % 100:02d}'

print('The average swim time for males, 45-50, in sprint triathlons is', formatted_swim_time, 'to go 750 meters')
print('The average cycle time for males, 45-50, in sprint triathlons is', formatted_cycle_time, 'to go 12.4 miles')
print('The average run time for males, 45-50, in sprint triathlons is', formatted_run_time, 'to go 3.1 miles')

The average swim time for males, 45-50, in sprint triathlons is 14:08 to go 750 meters
The average cycle time for males, 45-50, in sprint triathlons is 37:80 to go 12.4 miles
The average run time for males, 45-50, in sprint triathlons is 23:74 to go 3.1 miles
